# 🍎 Health Calculator Agent Tutorial 🍏

Welcome to the **Health Calculator Agent** tutorial, where we'll showcase how to:
1. **Initialize** a project and use the Azure AI Foundry ecosystem
2. **Create an Agent** with **Code Interpreter** capabilities
3. **Perform BMI calculations** and **analyze nutritional data** with sample CSV files
4. **Generate** basic health insights and disclaimers

### ⚠️ Important Medical Disclaimer ⚠️
> **The information provided by this notebook is for general educational and entertainment purposes only and is not intended as a substitute for professional medical advice, diagnosis, or treatment.** Always seek the advice of your physician or other qualified health provider with any questions you may have regarding a medical condition.

## Prerequisites
- Python 3.8+
- `azure-ai-projects` and `azure-ai-inference` (for advanced inference)
- `opentelemetry-sdk` and `azure-core-tracing-opentelemetry` (optional, for tracing)
- A `.env` file in the parent directory of this notebook containing:
  ```bash
  PROJECT_CONNECTION_STRING=<your-project-connection-string>
  MODEL_DEPLOYMENT_NAME=<your-model-deployment-name>
  TENANT_ID=<your-tenant-id>
  ```

## Let's Dive In
We'll walk step-by-step, similar to our **Fun & Fit** sample, but with a focus on using **Code Interpreter** for numeric calculations and data analysis. Let's begin!

<img src="./seq-diagrams/2-code-interpreter.png" width="30%"/>




## 1. Initial Setup
We'll start by importing libraries, loading environment variables, and initializing an **AIProjectClient**. We'll also create a sample CSV for demonstration.


In [ ]:
# Import required libraries
import os
import time
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool, FilePurpose, MessageTextContent

# Load environment variables from the parent directory's .env
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

# Initialize AIProjectClient
try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=os.environ["PROJECT_CONNECTION_STRING"],
    )
    print("✅ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"❌ Error initializing client: {str(e)}")

# Create sample CSV data for demonstration
def create_sample_data():
    try:
        data = {
            'Date': pd.date_range(start='2024-01-01', periods=7),
            'Calories': [2100, 1950, 2300, 2050, 1900, 2200, 2150],
            'Protein_g': [80, 75, 85, 78, 72, 82, 79],
            'Carbs_g': [250, 230, 270, 245, 225, 260, 255],
            'Fat_g': [70, 65, 75, 68, 63, 73, 71],
            'Fiber_g': [25, 22, 28, 24, 21, 26, 23]
        }
        df = pd.DataFrame(data)
        filename = "nutrition_data.csv"
        df.to_csv(filename, index=False)
        print(f"📄 Created sample data file: {filename}")
        return filename
    except Exception as e:
        print(f"❌ Error creating sample data: {e}")
        return None

sample_file = create_sample_data()

## 2. Create Health Calculator Agent 👩‍💻
We'll upload our sample CSV and then create an agent with **Code Interpreter** enabled. This agent can read the file, run Python code, and return results and visualizations.


In [ ]:
def create_health_calculator(file_path):
    """Create an agent with code interpreter for health/nutrition calculations."""
    try:
        uploaded_file = project_client.agents.upload_file_and_poll(
            file_path=file_path,
            purpose=FilePurpose.AGENTS
        )
        print(f"✅ Uploaded CSV file, ID: {uploaded_file.id}")

        # Create a Code Interpreter tool referencing the uploaded file
        code_tool = CodeInterpreterTool(file_ids=[uploaded_file.id])

        # Create the agent with instructions to do basic calculations
        agent = project_client.agents.create_agent(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini"),
            name="health-calculator-agent",
            instructions="""
            You are a health calculator agent that can:
            1. Calculate and interpret BMI
            2. Analyze provided nutrition data
            3. Generate charts/plots
            4. Include disclaimers that you are not a medical professional
            """,
            tools=code_tool.definitions,
            tool_resources=code_tool.resources
        )
        print(f"🎉 Created health calculator agent, ID: {agent.id}")
        return agent, uploaded_file
    except Exception as e:
        print(f"❌ Error creating health calculator agent: {e}")
        return None, None

health_agent, uploaded_file = None, None
if sample_file:
    health_agent, uploaded_file = create_health_calculator(sample_file)

## 3. BMI Calculation with Code Interpreter
We'll create a thread for BMI calculations. We'll feed in the user's height/weight, and ask the agent to show how it calculates BMI, interpret the result, and always disclaim professional advice.


In [ ]:
def calculate_bmi_with_agent(agent, height_inches, weight_pounds):
    """Calculate BMI using the code interpreter agent."""
    try:
        # Create a new conversation thread
        thread = project_client.agents.create_thread()
        print(f"📝 Created thread for BMI calculation, ID: {thread.id}")

        # Construct user message requesting BMI calculation
        user_text = (
            f"Calculate BMI for \n"
            f"Height: {height_inches} inches\n"
            f"Weight: {weight_pounds} pounds\n"
            "Please: \n"
            "1. Show calculation \n"
            "2. Interpret the result \n"
            "3. Include disclaimers \n"
        )

        msg = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=user_text
        )
        print(f"➕ Created BMI request message, ID: {msg.id}")

        # Create and process the run, letting the agent handle code
        run = project_client.agents.create_and_process_run(
            thread_id=thread.id,
            assistant_id=agent.id
        )
        print(f"🤖 BMI run finished with status: {run.status}")
        return thread, run
    except Exception as e:
        print(f"❌ Error during BMI calculation: {e}")
        return None, None

if health_agent:
    bmi_thread, bmi_run = calculate_bmi_with_agent(health_agent, 70, 180)  # example: 5'10" and 180 lbs


## 4. Nutrition Analysis
We'll create another thread where the user can ask the agent to analyze the **`nutrition_data.csv`** we uploaded. The agent can read the file, compute macros, produce charts, and disclaim that it's not offering personalized medical advice.


In [ ]:
def analyze_nutrition_data(agent):
    """Ask the agent to analyze the uploaded nutrition data."""
    try:
        thread = project_client.agents.create_thread()
        print(f"📝 Created thread for nutrition analysis, ID: {thread.id}")

        user_text = (
            "Analyze the CSV file with daily nutrition data.\n"
            "1. Compute average daily macros (calories, protein, carbs, fat, fiber).\n"
            "2. Create a chart to show trends.\n"
            "3. Discuss any insights or disclaimers.\n"
        )

        msg = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=user_text
        )
        print(f"➕ Created nutrition request message, ID: {msg.id}")

        run = project_client.agents.create_and_process_run(
            thread_id=thread.id,
            assistant_id=agent.id
        )
        print(f"🤖 Nutrition run finished with status: {run.status}")
        return thread, run
    except Exception as e:
        print(f"❌ Error analyzing nutrition data: {e}")
        return None, None

if health_agent:
    nutrition_thread, nutrition_run = analyze_nutrition_data(health_agent)

## 5. Viewing Results & Visualizations 📊
The agent may produce text insights, disclaimers, and even images with charts. Let's fetch them from our threads!


In [ ]:
def view_agent_responses(thread_id):
    try:
        messages = project_client.agents.list_messages(thread_id=thread_id)
        print("\n🔎 Agent Responses:")
        for msg in messages.data:
            if msg.role == "assistant" and msg.content:
                for c in msg.content:
                    if hasattr(c, "text"):
                        print("Response:", c.text.value, "\n")

        # If images were generated, let's save them
        for img in messages.image_contents:
            file_id = img.image_file.file_id
            outname = f"chart_{file_id}.png"
            project_client.agents.save_file(file_id=file_id, file_name=outname)
            print(f"🖼️ Saved image output: {outname}")

    except Exception as e:
        print(f"❌ Error viewing agent responses: {e}")

# Display BMI calculations
if bmi_thread and bmi_run:
    print("\n=== BMI Calculation Results ===")
    view_agent_responses(bmi_thread.id)

# Display nutrition analyses
if nutrition_thread and nutrition_run:
    print("\n=== Nutrition Analysis Results ===")
    view_agent_responses(nutrition_thread.id)

## 6. Cleanup & Best Practices
We can remove our agent and sample data if desired. In production, you might keep them for repeated usage.

### Best Practices in a Nutshell
1. **Data Handling** – Validate input data, handle missing values, properly manage file attachments.
2. **Calculations** – Provide formula steps, disclaimers, limit scope to general wellness, remind user you're not a doctor.
3. **Visualizations** – Use clear labeling and disclaimers that charts are for educational demonstrations.
4. **Security** – Monitor usage, limit access to code interpreter if dealing with proprietary data.


In [ ]:
def cleanup_all():
    try:
        # Delete the uploaded CSV file from the service
        if 'uploaded_file' in globals() and uploaded_file:
            project_client.agents.delete_file(uploaded_file.id)
            print("🗑️ Deleted uploaded file from agent service.")

        # Delete the agent if we created one
        if 'health_agent' in globals() and health_agent:
            project_client.agents.delete_agent(health_agent.id)
            print("🗑️ Deleted health calculator agent.")

        # Delete local CSV file
        if 'sample_file' in globals() and sample_file and os.path.exists(sample_file):
            os.remove(sample_file)
            print("🗑️ Deleted local sample CSV file.")

    except Exception as e:
        print(f"❌ Error during cleanup: {e}")

cleanup_all()

# Congratulations! 🎉
You now have a **Health Calculator Agent** with the **Code Interpreter** tool that can:
- Perform **BMI calculations** and disclaim that it's not a doctor.
- **Analyze** simple CSV-based nutrition data and produce insights + charts.
- Return images (charts) and text-based insights.

Feel free to extend this example to handle more advanced **file search**, **function calling**, or **evaluation** using the `azure-ai-evaluation` library!

> **Happy (healthy) coding!**
